# 数据聚合与分组计算

对数据集进行**分组**并对各组应用一个**函数**(无论是聚合还是转换),这是数据分析工作中的重要环节。在将数据集准备好之后,通常的任务就是**计算分组统计**或**生成透视表**。pandas提供了一个灵活高效的**gruopby**功能,它使你能以一种自然的方式对数据集进行**切片**、**切块**、**摘要**等操作。

关系型数据库和SQL(Structured Query Language,结构化查询语言)能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。但是,像SQL这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到,由于Python和pandas强大的表达能力,我们可以执行复杂得多的分组运算(利用任何可以接受pandas对象或NumPy数组的函数)。

在本章中,你将会学到:
* 根据一个或多个键(可以是函数、数组或DataFrame列名)拆分pandas对象。
* 计算分组摘要统计,如计数、平均值、标准差,或用户自定义函数。
* 对DataFrame的列应用各种各样的函数。
* 应用组内转换或其他运算,如规格化、线性回归、排名或选取子集等。
* 计算透视表或交叉表。
* 执行分位数分析以及其他分组分析。

## GroupBy

分组运算（split-apply-combine）：
* 拆分：
    * 通过一个或多个键对原数据进行拆分到不同组中；
* 应用：
    * 在不同组上应用函数计算得到结果；
* 合并：
    * 将结果合并到最终的结果对象中；

下图很好的展示了该过程：
![分组计算]()

In [2]:
!ls images

分组计算.png
